# Web Scraping Teil 2 (Lösungen)

☝️ Beachte: Es gibt beim Programmieren fast immer verschiedene Lösungswege. Deine Lösung mag anders aussehen, aber dennoch zum gewünschten Resultat führen. Das richtige Resultat ist das Wichtigste. 

⚠️ Führ folgenden Code aus, bevor Du einzelne Lösungen ausführst. 

In [ ]:
import requests
from bs4 import BeautifulSoup
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
import xml.etree.ElementTree as ET 

***

🔧 **Anwendungsfall (komplette Lösung):** 

Abrufschritt:

In [ ]:
import re
import time

#Definieren eines regulären Ausdrucks, um den Link zur jeweils nächsten Seite zu extrahieren
regex = r'href="(\S+)">weiter' #Runde Klammern umschließen Gruppe mit eigentlichem Link

#Definieren von Stammlink, erster Linkendung sowie des kompletten Links zur ersten zu scrapenden Seite
base_link = "https://www.projekt-gutenberg.org/goethe/faust1/"
link_ending = "chap002.html"
link = base_link + link_ending

all_pages = [] #Initialisieren einer leeren Liste, an die unten die einzelnen Quelltexte gehängt werden

"""'while'-Schleife scrapt eine paginierte Seite nach der anderen, bis kein Link mehr auf eine nächste Seite
im Quelltext gefunden wird"""
while True:
    
    print(f"Aktuell wird gescrapt: {link}") #Ausgabe des Fortschritts
    
    current_page = requests.get(link, timeout=5, headers=headers) #Abruf des Quelltexts zum aktuellen Link
    
    #Kontrolle des Statuscodes
    if not current_page.status_code == requests.codes.ok:
        print("Statuscode nicht ok!")
        break
    
    """Definieren des Encodings des Quelltexts (requests geht unspezifiziert vom falschen Encoding aus, 
    was sich z. B. an Umlauten zeigt)"""
    current_page.encoding = "UTF-8" 
    
    #Anfügen des eigentlichen Quelltexts der aktuellen Seite (Zugriff über text-Attribut) an 'all_pages'
    all_pages.append(current_page.text) 
    
    #Quelltext nach regulärem Ausdruck absuchen, der Link zur nächsten Seite matcht
    next_page = re.search(regex, current_page.text)
    
    #Wenn ein match gefunden wird...
    if next_page:
        """Definieren des Links für die nächste Seite, indem wir mithilfe der 
        'group'-Methode auf die erste Gruppe zugreifen und die neue Linkendung an den Stammlink hängen"""
        link = base_link + next_page.group(1)
        time.sleep(5) #Erst noch eine kleine Pause!
    else: #...wenn nicht, ist das Werk komplett gescrapet und die 'while'-Schleife wird abgebrochen.
        break

Extraktionsschritt:

In [ ]:
#Öffnen der Datei, in die die Strophen geschrieben werden sollen
with open("../../3_Dateien/Output/Faust_1.txt" , "w") as write_file: 
    
    for page in all_pages: #Iteration über 'all_pages'
 
        soup = BeautifulSoup(page, "lxml") #Konstruieren eines BeautifulSoup-Objekts
        body = soup.find("body") #Zugriff auf das body-Element über 'find'-Methode
        
        """Iteration über Liste mit allen Elementen mit Tag <p>, die, wie ein Blick in die Quelltexte ergab,
        die Strophen und Figuren (nebst weiteren Inhalten) enthält"""
        for paragraph in body.find_all("p"): 
            """Nun müssen wir diejenigen Elemente aus allen <p>-Elementen extrahieren, die die Figur bzw. 
            die Strophen enthalten."""
            
            """Vor jeder Strophe steht die Figur, die spricht, und zwar in einem dem <p>-Element
            untergeordneten Element mit <span>-Tag und Attribut class="speaker". Wir überprüfen mittels 
            'if'-Bedingung, ob die 'find'-Methode ein solches Element findet, wenn ja..."""
            if paragraph.find("span", class_="speaker"):
                """...extrahieren wir den darin enthaltenen Text... (Zugriff über 'text'-Attribut würde Fehlermeldung
                hervorrufen, wenn 'find' kein entsprechendes Element finden würde, daher if-Bedingung)"""
                speaker = paragraph.find("span", class_="speaker").text
                """...und schreiben ihn von inkonsistent verwendeten Doppelpunkten bereinigt 
                und mit abschließenden Zeilenumbrüchen in die Textdatei"""
                write_file.write(speaker.strip(":") + "\n\n") 
           
            elif paragraph.get("class") == ["vers"]:  #Da <p>-Elemente nichts weiter enthalten, wenn sie die Figur enthalten, 
                                                      #können wir mittels 'elif' überprüfen, ob das Attribut "class" ["vers"] 
                                                      #entspricht, denn in <p>-Elementen mit diesem Attribut sind 
                                                      #die Strophen enthalten
                
                """Stropheninterne Regieanweisungen befinden sich in einem <span>-Element, dessen Existenz
                im aktuelle <p>-Element wir hier überprüfen"""
                if paragraph.span:
                    paragraph.span.decompose() #Wenn ja, löschen wir es aus 'paragraph' mittels der 'decompose'-Methode
            
                """Da Einrückungen und whitespace generell in den Strophen inkonsistent verwendet werden,
                splitten wir die Strophen in Verse, um im 'write'-Befehl unten einheitlich formatieren zu können."""
                lines = paragraph.text.split("\n")
                
                for line in lines: #Iteration über die einzelnen Verse...
                    write_file.write("\t" + line.strip() + "\n") #...und schreiben in Textdokument (bereinigt und konsistent formatiert)
                
                write_file.write("\n") #Schreiben eines weiteren Zeilenumbruchs nach jeder Strophe (= zwei Zeilenumbrüche insgesamt)

***

🔧 **Anwendungsfall (Schritt-für-Schritt-Lösung):**

<u>Abruf</u>

1. Schau Dir die [erste Seite](https://www.projekt-gutenberg.org/goethe/faust1/chap002.html) des Werks auf www.projekt-gutenberg.org im Browser an und untersuch den ihr zugrundeliegenden Quelltext. Find so heraus, wie Du an die Links zu den folgenden Seiten kommst, damit Du auch diese scrapen kannst.

    *Lösung: Auf der ersten Seite, ebenso wie auf allen folgenden bis zur letzten, befindet sich eine Schaltfläche "Weiter". Im Quelltext siehst Du, dass sich dahinter der Link (bzw. die Linkendung) zur jeweils nächsten Seite verbirgt. Wir können folglich 1) die erste Seite scrapen, 2) deren Quelltext speichern, 3) darin nach dem Link zur nächsten Seite suchen, 4) die nächste Seite mithilfe des neuen Links ebenfalls scrapen und speichern usw., bis es keinen nächsten Link mehr gibt.*

    *Es gibt zwei weitere Alternativen: Erstens gibt es auf [dieser Seite](https://www.projekt-gutenberg.org/goethe/faust1/index.html) ein Inhaltsverzeichnis mit Links zu allen Szenen des Werks. Wir könnten also auch 1) diese Seite scrapen, 2) alle relevanten Links aus deren Quelltext extrahieren und in eine Liste überführen, 3) über die Liste iterieren und alle entsprechenden Quelltexte scrapen. Zweitens könnten wir uns den Umstand zu Nutze machen, dass die Seiten von zwei bis 28 durchnummeriert sind (vgl. Linkendungen). Mit der `range`-Funktion könnten wir einfach von zwei bis 28 durchiterieren und "on the fly" jeweils einen Link daraus basteln.*

    *In der Schritt-für-Schritt-Lösung verfolgen wir den ersten der drei Ansätze, da er sich am ehesten auf vergleichbare Anwendungsfälle mit paginierten Seiten übertragen lässt.*

2. Wie Du im Quelltext der ersten Seite erkennen kannst, verbirgt sich hinter der Schaltfläche "Weiter" nur jeweils die Linkendung, die um einen Stammlink ergänzt werden muss. 
    
    Definier den Stammlink in `base_link` sowie die Linkendung für die erste Seite in `link_ending`. Konkatenier die beiden strings zu `link` und lass Dir `link` ausgeben. Wenn Du auf den ausgegebenen Link klickst, solltest Du auf der ersten Seite des Werks landen.

In [ ]:
#Definieren von Stammlink, erster Linkendung sowie des kompletten Links zur ersten zu scrapenden Seite
base_link = "https://www.projekt-gutenberg.org/goethe/faust1/"
link_ending = "chap002.html"
link = base_link + link_ending
print(link)

3. Überleg Dir nun, mithilfe welcher Kontrollstruktur Du ausgehend von der ersten Seite nacheinander alle Seiten scrapen kannst, bis es keine weitere Seite mehr gibt. Welche weitere Kontrollstruktur kannst Du verwenden, um das Scraping in diesem Fall zu beenden? 

    *Lösung: Wir scrapen eine Seite nach der anderen mithilfe einer `while`-Schleife, konkret: `while True`. So bricht diese Schleife erst ab, wenn sie auf ein `break`-Statement trifft. Letzteres rücken wir unter der zweiten Kontrollstruktur, einer `if`-Bedingung, ein, die nur dann `True` ergeben soll, wenn der zuletzt gescrapte Quelltext **keinen** Link auf eine nächste Seite mehr enthält (also wenn wir auf der letzten Seite angelangt sind).*

4. Um von einer Seite zur nächsten zu gelangen, müssen wir ja jeweils den Link darauf aus der aktuellen Seite extrahieren. Definier dafür einen regulären Ausdruck, der den Link hinter der Schaltfläche "Weiter" matcht. Falls Du noch nicht mit regulären Ausdrücken vertraut bist, dann kopier den Code für diesen Schritt aus der Lösung.

In [ ]:
import re

#Definieren eines regulären Ausdrucks, um den Link zur jeweils nächsten Seite zu extrahieren
regex = r'href="(\S+)">weiter' #Runde Klammern umschließen Gruppe mit eigentlichem Link

5. Setz nun Deine Erkenntnisse aus Schritt 3 in Code um: 
    
    A. Schreib eine Schleife (erste Kontrollstruktur), die wiederholt wird, bis sie auf ein `break`-Statement trifft. 
    
    B. Verwend das `requests`-Moduls, um die Seite zum aktuellen `link` abzurufen (`link` entspricht ja zumindest am Anfang der ersten Seite, s.&nbsp;o.). Speichere das Response-Objekt in `current_page`.
    
    C. Da `requests` automatisch von einem falschen Encoding ausgeht (weswegen etwa Umlaute falsch dekodiert würden), müssen wir das Encoding korrigieren: `current_page.encoding = "UTF-8"`.
    
    D. Häng den eigentlichen Quelltext in `current_page` einer zuvor definierten Liste `all_pages` an, die sämtliche Quelltexte umfassen soll.
    
    E. Such den Quelltext nach `regex` ab. Verwend dazu die Funktion `search` des Moduls `re` (schau auch hier in der Lösung nach, wenn Du regulären Ausdrücken noch nicht vertraut bist).
   
    F. Überprüf nun mithilfe der zweiten Kontrollstruktur, ob ein match vorliegt. Wenn ja, überschreib `link`, indem Du den match, also die neue Linkendung, an `base_link` anhängst. Nun kann die nächste Seite gescrapt werden. Liegt kein match (mehr) vor, soll die Schleife abgebrochen werden.    

In [ ]:
import time

all_pages = [] #Initialisieren einer leeren Liste, an die unten die einzelnen Quelltexte gehängt werden

"""'while'-Schleife scrapt eine paginierte Seite nach der anderen, bis kein Link mehr auf eine nächste Seite
im Quelltext gefunden wird"""
while True:
    
    print(f"Aktuell wird gescrapt: {link}") #Ausgabe des Fortschritts
    
    current_page = requests.get(link, timeout=5, headers=headers) #Abruf des Quelltexts zum aktuellen Link
    
    #Kontrolle des Statuscodes
    if not current_page.status_code == requests.codes.ok:
        print("Statuscode nicht ok!")
        break
    
    """Definieren des Encodings des Quelltexts (requests geht unspezifiziert vom falschen Encoding aus, 
    was sich z. B. an Umlauten zeigt)"""
    current_page.encoding = "UTF-8" 
    
    #Anfügen des eigentlichen Quelltexts der aktuellen Seite (Zugriff über text-Attribut) an 'all_pages'
    all_pages.append(current_page.text) 
    
    #Quelltext nach regulärem Ausdruck absuchen, der Link zur nächsten Seite matcht
    next_page = re.search(regex, current_page.text)
    
    #Wenn ein match gefunden wird...
    if next_page:
        """Definieren des Links für die nächste Seite, indem wir mithilfe der 
        'group'-Methode auf die erste Gruppe zugreifen und die neue Linkendung an den Stammlink hängen"""
        link = base_link + next_page.group(1)
        time.sleep(5) #Erst noch eine kleine Pause!
    else: #...wenn nicht, ist das Werk komplett gescrapet und die 'while'-Schleife wird abgebrochen.
        break

<u>Extraktion</u>

6. Nun haben wir alle Quelltexte gescrapt und es geht ans Extrahieren der relevanten Daten. Wir schreiben zunächst Code, um alle Strophen sowie die Figuren aus einem *einzelnen* Quelltext zu extrahieren. Anschließend bauen wir diesen Code in eine Schleife, der über *sämtliche* Quelltexte iteriert und nach und nach das gesamte Werk extrahiert.

    Analysiere als Erstes die Quelltexte eingehend, entweder im Browser, in Sublime Text oder indem Du sie Dir mithilfe von `prettify` von BeautifulSoup ausgeben lässt. Welche Elemente mit welchen Tags und ggf. welchen Attributen beinhalten die Strophen und Figuren?
    
    *Lösung: Sowohl Strophen als auch Figuren sind in Elementen mit Tag `<p>` enthalten. Figuren sind in den `<p>`-Elementen **untergeordneten** `<span>`-Elementen mit Attribut `class="speaker"` enthalten. Die Strophen befinden sich in `<p>`-Elementen mit dem Attribut `class="vers"`.*

7. Verwend BeautifulSoup, um den ersten Quelltext zu parsen. Schaffe ein Objekt `body`, das nur noch das `<body>`-Element beinhält.

In [ ]:
page = all_pages[0] #Definieren des ersten Quelltexts als 'page'

soup = BeautifulSoup(page, "lxml") #Konstruieren eines BeautifulSoup-Objekts
body = soup.find("body") #Zugriff auf das body-Element über 'find'-Methode

8. Iterier mithilfe von `find_all` über alle Elemente desjenigen Tags, das sowohl Strophen als auch Figuren umfasst. 

    Überprüf für jedes Element erstens, ob sich darin untergeordnet dasjenige Element befindet, das die Figuren enthält. Wenn ja, extrahier es und weis es `speaker` zu. Bereinige `speaker` von überflüßigen Zeichen und lass es Dir ausgeben.

    Überprüf zweitens, ob das Element andernfalls über dasjenige Attribut verfügt, das sämtliche Elemente, die Strophen beinhalten, miteinander teilen. Wenn ja, extrahier es, unterteil es in Verse und weis diese `lines` zu. Lass Dir `lines` schön formatiert ausgeben
    
    ⚠️ Achtung: Manche Strophen beinhalten auch Regienanweisungen in einem untergeordneten `<span>`-Element. Bau folgenden Code an der richtigen Stelle ein, um diese `<span>`-Elemente aus den Strophen zu entfernen:
    
    `if line.span:`
     <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`line.span.decompose()`
    
    Wir überprüfen damit erst, ob die entsprechende Strophe ein `<span>`-Element enthält und wenn ja, entfernen wir es aus ihr mithilfe der `decompose`-Methode. 

In [ ]:
"""Iteration über Liste mit allen Elementen mit Tag <p>, die, wie ein Blick in die Quelltexte ergab,
die Strophen und Figuren (nebst weiteren Inhalten) enthält"""
for paragraph in body.find_all("p"): 
    """Nun müssen wir diejenigen Elemente aus allen <p>-Elementen extrahieren, die die Figur bzw. 
    die Strophen enthalten."""

    """Vor jeder Strophe steht die Figur, die spricht, und zwar in einem dem <p>-Element
    untergeordneten Element mit <span>-Tag und Attribut class="speaker". Wir überprüfen mittels 
    'if'-Bedingung, ob die 'find'-Methode ein solches Element findet, wenn ja..."""
    if paragraph.find("span", class_="speaker"):
        """...extrahieren wir den darin enthaltenen Text... (Zugriff über 'text'-Attribut würde Fehlermeldung
        hervorrufen, wenn 'find' kein entsprechendes Element finden würde, daher if-Bedingung)"""
        speaker = paragraph.find("span", class_="speaker").text
        """...und schreiben ihn von inkonsistent verwendeten Doppelpunkten bereinigt 
        und mit abschließenden Zeilenumbrüchen in die Textdatei"""
        print(speaker.strip(":") + "\n") 

    elif paragraph.get("class") == ["vers"]:  #Da <p>-Elemente nichts weiter enthalten, wenn sie die Figur enthalten, 
                                              #können wir mittels 'elif' überprüfen, ob das Attribut "class" ["vers"] 
                                              #entspricht, denn in <p>-Elementen mit diesem Attribut sind 
                                              #die Strophen enthalten

        """Stropheninterne Regieanweisungen befinden sich in einem <span>-Element, dessen Existenz
        im aktuelle <p>-Element wir hier überprüfen"""
        if paragraph.span:
            paragraph.span.decompose() #Wenn ja, löschen wir es aus 'paragraph' mittels der 'decompose'-Methode

        """Da Einrückungen und whitespace generell in den Strophen inkonsistent verwendet werden,
        splitten wir die Strophen in Verse, um im 'write'-Befehl unten einheitlich formatieren zu können."""
        lines = paragraph.text.split("\n")
        
        for line in lines: #Iteration über die einzelnen Verse...
            print("\t" + line.strip())
        
        print("\n")

9. Modifizier den Code aus Schritt 8 derart, dass Du die relevanten Daten nicht nur aus *einem* Quelltext extrahierst, sondern aus *allen* auf `all_pages`. Pass ihn außerdem so an, dass Dir Figuren und Strophen nicht ausgegeben werden, sondern dass diese in eine externe Datei geschrieben werden. 

In [ ]:
#Öffnen der Datei, in die die Strophen geschrieben werden sollen
with open("../../3_Dateien/Output/Faust_1.txt" , "w") as write_file: 
    
    for page in all_pages: #Iteration über 'all_pages'
 
        soup = BeautifulSoup(page, "lxml") #Konstruieren eines BeautifulSoup-Objekts
        body = soup.find("body") #Zugriff auf das body-Element über 'find'-Methode
        
        """Iteration über Liste mit allen Elementen mit Tag <p>, die, wie ein Blick in die Quelltexte ergab,
        die Strophen und Figuren (nebst weiteren Inhalten) enthält"""
        for paragraph in body.find_all("p"): 
            """Nun müssen wir diejenigen Elemente aus allen <p>-Elementen extrahieren, die die Figur bzw. 
            die Strophen enthalten."""
            
            """Vor jeder Strophe steht die Figur, die spricht, und zwar in einem dem <p>-Element
            untergeordneten Element mit <span>-Tag und Attribut class="speaker". Wir überprüfen mittels 
            'if'-Bedingung, ob die 'find'-Methode ein solches Element findet, wenn ja..."""
            if paragraph.find("span", class_="speaker"):
                """...extrahieren wir den darin enthaltenen Text... (Zugriff über 'text'-Attribut würde Fehlermeldung
                hervorrufen, wenn 'find' kein entsprechendes Element finden würde, daher if-Bedingung)"""
                speaker = paragraph.find("span", class_="speaker").text
                """...und schreiben ihn von inkonsistent verwendeten Doppelpunkten bereinigt 
                und mit abschließenden Zeilenumbrüchen in die Textdatei"""
                write_file.write(speaker.strip(":") + "\n\n") 
           
            elif paragraph.get("class") == ["vers"]:  #Da <p>-Elemente nichts weiter enthalten, wenn sie die Figur enthalten, 
                                                      #können wir mittels 'elif' überprüfen, ob das Attribut "class" ["vers"] 
                                                      #entspricht, denn in <p>-Elementen mit diesem Attribut sind 
                                                      #die Strophen enthalten
                
                """Stropheninterne Regieanweisungen befinden sich in einem <span>-Element, dessen Existenz
                im aktuelle <p>-Element wir hier überprüfen"""
                if paragraph.span:
                    paragraph.span.decompose() #Wenn ja löschen wir es aus 'paragraph' mittels der 'decompose'-Methode
            
                """Da Einrückungen und whitespace generell in den Strophen inkonsistent verwendet werden,
                splitten wir die Strophen in Verse, um im 'write'-Befehl unten einheitlich formatieren zu können."""
                lines = paragraph.text.split("\n")
                
                for line in lines: #Iteration über die einzelnen Verse...
                    write_file.write("\t" + line.strip() + "\n") #...und schreiben in Textdokument (bereinigt und konsistent formatiert)
                
                write_file.write("\n") #Schreiben eines weiteren Zeilenumbruchs nach jeder Strophe (= zwei Zeilenumbrüche insgesamt)

***

✏️ **Übung 1:** Pass obigen Extraktionscode so an, dass vor jeder Äußerung der Name des/der jeweiligen Abgeordneten ausgegeben wird. 

In [ ]:
#Parsen des XML-Dokuments sowie Extrahieren von 'root' (nur im Lösungsnotebook notwendig)
#Achtung: anderer Pfad als im Notebook, da das Lösungsnotebook in einem anderen Verzeichnis liegt 
tree = ET.parse("../../3_Dateien/XML/plenarprotokoll.xml")
root = tree.getroot()

for speech in root.iter("rede"):
    
    """Eigentliche Lösung (Anfang): Wir extrahieren Vor- und Nachnamen, indem wir 'speech' mithilfe von 'iter' 
    rekursiv nach den gewünschten Tags absuchen, das Resultat in eine Liste casten, das erste Element davon indizieren 
    und auf dessen Textinhalt zugreifen. Anstatt Casting in Liste und Indizierung wäre auch jeweils eine 'for'-Schleife 
    möglich, was den Code aber länger machen würde. Alternativ könnten wir auch die RegEx-ähnliche Suchsprache XPath
    verwenden, was für die Extraktion von 'name' so aussähe: name = speech.find(".//p[@klasse='redner']//vorname").text
    vgl. dazu die Dokumentation von XPath: https://www.w3.org/TR/xpath-31/"""
    
    name = list(speech.iter("vorname"))[0].text
    surname = list(speech.iter("nachname"))[0].text
    print(name, surname, sep=" ") #Ausgabe des konkatenierten Namens
    
    """Eigentliche Lösung (Ende)"""
    
    for element in speech:
        if element.tag == "name":
            break
        elif element.tag == "p" and element.text:
            print(element.text.strip(), end=" ") #Inkl. Bereinigung von leading/trailing whitespace
    print("\n") #Einfügen eines Zeilenumbruchs nach jeder Rede

***

✏️ **Übung 2:** Neben den Reden bzw. Äußerungen der Abgeordneten, die gerade das Rederecht besitzen, enthält das Protokoll auch verbale Zwischenrufe sowie Anmerkungen über Beifall, Lachen etc. Diese Informationen sind in Elementen mit dem Tag `<kommentar>` enthalten. Extrahier sie für alle Reden, für die mindestens ein `<kommentar>`-Element protokolliert wurde. Lass sie Dir zusammen mit dem Namen des/der Abgeordneten ausgeben, der/die eigentlich gerade am sprechen ist.

In [ ]:
#Achtung: Code funktioniert nur, wenn der Code zur vorangehenden Übung ausgeführt wurde!

for speech in root.iter("rede"):
    
    #Überprüfung, ob es überhaupt <kommentar>-Elemente gibt, wenn nein, überspringen
    if len(list(speech.iter("kommentar"))) == 0:
        continue 

    #Erläuterung vgl. Lösung zur ersten Übung
    name = list(speech.iter("vorname"))[0].text
    surname = list(speech.iter("nachname"))[0].text
    print(name, surname, sep=" ")
    
    #Rekursives Iterieren über 'speech' und Suche nach 'kommentar'-Elementen
    for comment in speech.iter("kommentar"):
        print(comment.text) #Ausgabe des Textinhalts
    
    print("\n")

*** 

✏️ **Übung 3:** Im Anwendungsfall haben wir sämtliche Strophen von Faust I in einer externen Textdatei gespeichert. Diese verfügt (idealerweise) über eine interne Struktur. In der Musterlösung wurde etwa mit Tabstopps und Zeilenumbrüchen gearbeitet, um einzelne Strophen bzw. die zugehörigen Figuren voneinander abzugrenzen. Diese Struktur kommt uns spätestens dann zu Gute, wenn wir die Daten für irgendeine Form von Auswertung mit Python wieder einlesen. 

Um unseren Anwendungsfall zu "professionalisieren", ist es nun Deine Aufgabe, den Faust I in einem XML-Dokument speichern, also in einem Format, dessen "Aufgabe" es ist, Daten ordentlich zu strukturieren. Kopier dazu den Code vom Extraktionsschritt des Anwendungsfalls in die folgende Zelle. Pass ihn anschließend so an, dass die Daten dynamisch in ein XML-Dokument statt in eine Textdatei geschrieben werden.

Als kleiner Bonus, der überprüft, ob bei der Speicherung alles geklappt hat, vor allem aber die Nützlichkeit strukturierter Daten aufzeigt, kannst Du Dir anschließend über die bereits gegebene Code-Zelle einfach alle Verse des berühmten Teufels aus dem Faust ausgeben lassen. Wenn nötig, pass den Dateipfad bzw. die Tags und Attribute Deiner Namensgebung an.

<details><summary>🦊 Herausforderung </summary>
<br>Extrahier zusätzlich den Titel jeder Szene und speichere diese Information jeweils an der richtigen Position in der Hierarchie des zu schaffenden XML-Dokuments.
</details>

In [ ]:
"""Alle Kommentare beziehen sich auf den neu hinzugekommenen Lösungscode; Kommentare zum Extraktionscode
siehe Anwendungsfall oben; Code funktioniert nur, wenn 'all_pages' vom Abrufschritt noch im Arbeitsspeicher ist!"""

from xml.dom import minidom

"""Initialisieren des obersten Elements in der Hierarchie. Elemente können in XML ja frei benannt werden.
Die Text Encoding Initiative (TEI; https://www.tei-c.org) strebt eine Standardisierung der Repräsentation
und Speicherung von Texten an und bietet auch genrespezifische Empfehlungen (hier für Dramen: 
https://www.tei-c.org/release/doc/tei-p5-doc/en/html/DR.html). Im Folgenden wurden diese 
Empfehlungen nur lose umgesetzt."""
text = ET.Element("text") 

#Optionales Hinzufügen von Attributen
text.set("title", "Faust I")
text.set("source", "https://www.projekt-gutenberg.org/goethe/faust1/index.html") 

for page in all_pages:

    soup = BeautifulSoup(page, "lxml") 
    body = soup.find("body") 

    for paragraph in body.find_all("p"):

        if paragraph.find("span", class_="speaker"):
            
            """Initialisieren eines Kindelements von 'text', das sämtliche aufeinanderfolgende Strophen
            einer Figur beinhalten soll"""
            character = ET.SubElement(text, "character")
            
            speaker = paragraph.find("span", class_="speaker").text
            
            character.set("name", speaker.strip(":")) #Setzen eines Attributs, das den Namen der Figur beinhaltet

        elif paragraph.get("class") == ["vers"]: 
            
            """Initialisieren eines Kindelements von 'character', das sämtliche Verse einer (!)
            Strophe beinhalten soll"""
            stanza = ET.SubElement(character, "stanza")

            if paragraph.span:
                paragraph.span.decompose() 

            lines = paragraph.text.split("\n")

            for line in lines: 
                
                """Initialisieren eines Kindelements von 'stanza', das jeweils einen Vers
                beinhalten soll. Achtung: die Variable 'line' (die 'Vers' bedeutet) ist bereits 
                in Benutzung. Daher weisen wir das Kindelement der Variablen 'line_xml' zu.
                Das XML-Element heißt dennoch nur 'line'. Es zeigt sich, dass Variablen, die ein 
                XML-Element referenzieren, unabhängig von dessen Tag (Name) benannt werden können."""
                line_xml = ET.SubElement(stanza, "line")
                line_xml.text = line.strip()

#Hinzufügen von Einrückungen
pretty_corpus = minidom.parseString(ET.tostring(text)).toprettyxml(indent="  ")

#Schreiben von 'pretty_corpus' in externe Datei mit der Endung 'xml' (anderer Pfad als im Notebook!)
with open("../../3_Dateien/Output/faust_1.xml", "w", encoding="utf-8") as write_file:
    write_file.write(pretty_corpus)

In [ ]:
#Bonus: Ausgabe aller Verse von Mephisto(pheles)
root = ET.parse("../../3_Dateien/Output/faust_1.xml").getroot() #Anderer Pfad als im Notebook!

for character in root.iter("character"):
    if not character.get("name") == "Mephistopheles":
        continue
    for line in character.iter("line"):
        print(line.text)

In [ ]:
#Herausforderung

"""Alle Kommentare beziehen sich auf den neu hinzugekommenen Lösungscode; Kommentare zum Extraktionscode
siehe Anwendungsfall oben; Code funktioniert nur, wenn 'all_pages' vom Abrufschritt noch im Arbeitsspeicher ist!"""

from xml.dom import minidom

"""Initialisieren des obersten Elements in der Hierarchie. Elemente können in XML ja frei benannt werden.
Die Text Encoding Initiative (TEI; https://www.tei-c.org) strebt eine Standardisierung der Repräsentation
und Speicherung von Texten an und bietet auch genrespezifische Empfehlungen (hier für Dramen: 
https://www.tei-c.org/release/doc/tei-p5-doc/en/html/DR.html). Im Folgenden wurden diese 
Empfehlungen nur lose umgesetzt."""
text = ET.Element("text") 

#Optionales Hinzufügen von Attributen
text.set("title", "Faust I")
text.set("source", "https://www.projekt-gutenberg.org/goethe/faust1/index.html")

for page in all_pages:

    soup = BeautifulSoup(page, "lxml") 
    body = soup.find("body") 
    
    scene = soup.find("h3").text #Zusätzliche Extraktion des Szenentitels
    
    """Um Faust I nach Szenen zu gliedern, drängt sich die Schaffung einer weiteren hierarchischen Ebene
    zwischen 'faust' und 'character' auf. Deshalb initialisieren wir hier ein Kindelement von 'text', 
    das sämtliche Strophen einer Szene beinhalten soll. Der Szenentitel soll in einem Attribut gespeichert werden."""
    scene = ET.SubElement(text, "scene", {"title": scene})
    
    for paragraph in body.find_all("p"):

        if paragraph.find("span", class_="speaker"):
            
            """Initialisieren eines Kindelements von 'scene', das sämtliche aufeinanderfolgende Strophen
            einer Figur beinhalten soll"""
            character = ET.SubElement(scene, "character")
            
            speaker = paragraph.find("span", class_="speaker").text
            
            character.set("name", speaker.strip(":")) #Setzen eines Attributs, das den Namen der Figur beinhaltet
            
        elif paragraph.get("class") == ["vers"]: 
            
            """Initialisieren eines Kindelements von 'character', das sämtliche Verse einer (!)
            Strophe beinhalten soll"""
            stanza = ET.SubElement(character, "stanza")

            if paragraph.span:
                paragraph.span.decompose() 

            lines = paragraph.text.split("\n")

            for line in lines: 
                
                """Initialisieren eines Kindelements von 'stanza', das jeweils einen Vers
                beinhalten soll. Achtung: die Variable 'line' (die 'Vers' bedeutet) ist bereits 
                in Benutzung. Daher weisen wir das Kindelement der Variablen 'line_xml' zu.
                Das XML-Element heißt dennoch nur 'line'. Es zeigt sich, dass Variablen, die ein 
                XML-Element referenzieren, unabhängig von dessen Tag (Name) benannt werden können."""
                line_xml = ET.SubElement(stanza, "line")
                line_xml.text = line.strip()

#Hinzufügen von Einrückungen
pretty_corpus = minidom.parseString(ET.tostring(text)).toprettyxml(indent="  ")

#Schreiben von 'pretty_corpus' in externe Datei mit der Endung 'xml' (anderer Pfad als im Notebook!)
with open("../../3_Dateien/Output/faust_1_mit_szenentitel.xml", "w", encoding="utf-8") as write_file:
    write_file.write(pretty_corpus)

In [ ]:
#Bonus: Ausgabe aller Verse von Mephisto(pheles)
root = ET.parse("../../3_Dateien/Output/faust_1_mit_szenentitel.xml").getroot() #Anderer Pfad als im Notebook!

for character in root.iter("character"):
    if not character.get("name") == "Mephistopheles":
        continue
    for line in character.iter("line"):
        print(line.text)

***

✏️ **Übung 4:** XML ist wie gesagt ein beliebtes Format, um Daten zu speichern und zu teilen. Neben XML arbeiten wir beim Programmieren auch oft mit csv-Dateien (vgl. Notebooks "Input und Output" sowie "Datenanalyse"). Welchen Vorteil hat XML im Gegensatz zu csv?

*Lösung: Informationen, die bei XML **einmal** gespeichert sind (hier sämtliche Informationen außer die Wörter der Rede), müssen bei csv für jedes Element auf der tiefsten Ebene (also für jedes Wort) **wiederholt** werden. Das liegt an der fehlenden Hierarchie bei csv-Dateien. XML spart deshalb Speicherplatz. Allerdings können csv-Dateien aufgrund der häufig wiederholten, identischen Informationen sehr stark komprimiert werden. Der Vorteil von XML besteht also nur, wenn man aktiv mit den Daten arbeitet. Zum Speichern bzw. Teilen können csv-Dateien ähnlich kompakt gemacht werden wie XML-Dateien.*


***
<table>
      <tr>
        <td>
            <img src="../../3_Dateien/Lizenz/CC-BY-SA.png" width="400">
        </td> 
        <td>
            <p>Dieses Notebook sowie sämtliche weiteren <a href="https://github.com/yannickfrommherz/exdimed-student/tree/main">Materialien zum Programmierenlernen für Geistes- und Sozialwissenschaftler:innen</a> sind im Rahmen des Projekts <i>Experimentierraum Digitale Medienkompetenz</i> als Teil von <a href="https://tu-dresden.de/gsw/virtuos/">virTUos</a> entstanden. Erstellt wurden sie von Yannick Frommherz unter Mitarbeit von Anne Josephine Matz. Sie stehen als Open Educational Resource nach <a href="https://creativecommons.org/licenses/by-sa/4.0/">CC BY SA</a> zur freien Verfügung. Für Feedback und bei Fragen nutz bitte das <a href="https://forms.gle/VsYJgy4bZTSqKioA7">Kontaktformular</a>.
        </td>
      </tr>
</table>